In [1]:
# do this step below to get lightning, lightning bolts, etc.
# !pip install lightning-bolts torchvision torchsummary

# Classification Revisited, with ResNets

*AY 128/256 (UC Berkeley, 2018–2024)*

Previously, we used CNNs in our own custom model to classify images. You are asked to stack up your own model to classify galaxies in Lab 3, and you should definitely explore different CNN depths, kernel sizes, and filters to get a feel for how this works. Many of them will be able to perform the functions you need for Lab 3 adequately. That said, some architectures are better than others, and an architecture we are going to introduce today, **ResNets (Residual neural Networks)** are particularly efficient at image classification.

More generally, PyTorch comes with a bunch of models and pre-trained weights that come from fitting to some generic (in this case classification) data sets. As it turns out, many of the features in these trained CNNs are widely applicable and can be easily repurposed for another task. In this lecture, we will explore how to do that.

Again, we'll make use of the Fashion MNIST labeled dataset, which you may recall looked something like this:

<img src="https://github.com/zalandoresearch/fashion-mnist/blob/master/doc/img/fashion-mnist-sprite.png?raw=true" width="80%">

With labels: 

In [2]:
lookup = {0: "T-shirt/top",
          1: "Trouser",
          2: "Pullover",
          3: "Dress",
          4: "Coat",
          5: "Sandal",
          6: "Shirt",
          7: "Sneaker",
          8: "Bag",
          9: "Ankle boot"}

def output_label(label):
    input = (label.item() if type(label) == torch.Tensor else label)
    return lookup[input]

nb_classes = len(lookup)

In [3]:
import datetime, os
import numpy as np
import warnings
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from torchmetrics.functional import accuracy
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint, ModelSummary
from pytorch_lightning.loggers import CSVLogger

import torchvision
from torchvision import models, transforms
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix

import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision.datasets import FashionMNIST
from torch.utils.data import random_split, DataLoader
import time
import copy

# use a GPU or MPS (Mac) if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.backends.mps.is_available():
    device = "mps"

print("pytorch version:", torch.__version__)
print("device:", device)

pytorch version: 2.3.0+cu121
device: cuda:0


In [4]:
train_csv = pd.read_csv("../Lecture20_lab3_pytorch/fashion-mnist_train.csv.gz")
test_csv = pd.read_csv("../Lecture20_lab3_pytorch/fashion-mnist_test.csv.gz")

In [5]:
%%writefile fashion_dataset.py

import numpy as np
from torch.utils.data import Dataset

class FashionDataset(Dataset):
    """User defined class to build a datset using Pytorch class Dataset."""
    
    def __init__(self, data, transform = None):
        """Method to initilaize variables.""" 
        self.fashion_MNIST = list(data.values)
        self.transform = transform
        
        label = []
        image = []
        
        for i in self.fashion_MNIST:
             # first column is of labels.
            label.append(i[0])
            image.append(i[1:])
        self.labels = np.asarray(label)
        # Dimension of Images = 28 * 28 * 1. where height = width = 28 and color_channels = 1.
        self.images = np.asarray(image).reshape(-1, 28, 28, 1).astype('float32')

    def __getitem__(self, index):
        label = self.labels[index]
        image = self.images[index]
        
        if self.transform is not None:
            image = self.transform(image)

        return image, label

    def __len__(self):
        return len(self.images)

Overwriting fashion_dataset.py


In [6]:
from fashion_dataset import FashionDataset

In [7]:
batch_size = 128

## Transform data into Tensor that has a range from 0 to 1
data_transforms = {
    'train': transforms.Compose([
        torchvision.transforms.ToPILImage(),
        torchvision.transforms.RandomAffine(degrees=15, shear=0.1),
        #transforms.Resize(28),
        transforms.RandomHorizontalFlip(),
        transforms.Grayscale(3), 
        transforms.ToTensor(), 
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
    'test': transforms.Compose([
        torchvision.transforms.ToPILImage(),
        torchvision.transforms.RandomAffine(degrees=15, shear=0.1),
        #transforms.Resize(28),
        transforms.Grayscale(3),
        transforms.ToTensor(), 
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])
}

train_set = FashionDataset(train_csv, transform=data_transforms['train'])
test_set = FashionDataset(test_csv, transform=data_transforms['test'])

train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=2)
test_loader = DataLoader(train_set, batch_size=batch_size, num_workers=2)

train_label = torch.tensor([train_set[i][1] for i in range(nb_classes)])

## Our (Previous) CNN

Let's inspect the model we created:

In [8]:
class mycnn_dropout(pl.LightningModule):

    def __init__(self):
        super().__init__()

        # define the layers here
        # Conv2d(in_channels, out_channels, kernel_size)
        # see https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html
        self.layer1 = nn.Sequential(
            #nn.Conv2d(1, 32, kernel_size=3),
            nn.Conv2d(3, 32, kernel_size=3),
            
            # see https://github.com/sksq96/pytorch-summary/issues/55#issuecomment-471844028
            # to understand why pytorch and keras differ here
            nn.BatchNorm2d(32, affine=False),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(p=0.1)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3),
            nn.ReLU(),
        )
        
        self.fc1=torch.nn.Linear(1152, 32)
        self.fc2=torch.nn.Linear(32, 10)
    
        self.loss = nn.NLLLoss()
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # add dropout 
        x = nn.Dropout(p=0.2)(x)

        x=torch.relu(self.fc1(x))
        x=F.log_softmax(self.fc2(x), dim=-1)
        return x

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters())
        
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode='min',
            factor=0.75,
            patience=2,
            min_lr=1e-6,
            verbose=True
        )
        
        return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "val_accuracy"}
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = self.loss(logits, y)
        self.log('train_loss', loss)
        return loss
    
    def _evaluate(self, batch, batch_idx, stage=None):
        x, y = batch
        logits = self.forward(x)
        loss = self.loss(logits, y)
        preds = torch.argmax(logits, dim=-1)
        acc = accuracy(preds, y, task="multiclass", num_classes=nb_classes)

        if stage:
            self.log(f'{stage}_loss', loss, prog_bar=True)
            self.log(f'{stage}_accuracy', acc, prog_bar=True)

        return loss, acc
    
    def validation_step(self, batch, batch_idx):
        return self._evaluate(batch, batch_idx, 'val')[0]
    
    def train_dataloader(self):
        return train_loader
    
    def val_dataloader(self):
        return test_loader

In [9]:
run_time_string = datetime.datetime.utcnow().isoformat(timespec='minutes')
filename = f'datalab_nn_pytorch_dropout_{run_time_string}'

early_stop_callback = EarlyStopping(
   monitor='val_accuracy',
   min_delta=0.001,
   patience=3,
   verbose=True,
   mode='max'
)

checkpoint_callback = ModelCheckpoint(
    monitor='val_accuracy',
    mode='max',
    dirpath='nn_results',
    filename=filename,
    verbose=True,
    save_top_k=1
)

logger = [CSVLogger("nn_results1", name=filename), TensorBoardLogger("nn_results1", name=filename)]

pl.seed_everything(42)

if device.type == "cuda":
    myTrainer=pl.Trainer(callbacks=[early_stop_callback, checkpoint_callback], logger=logger,
                     gpus=-1, accelerator='cuda', auto_select_gpus=True, max_epochs=5)
else:
    myTrainer=pl.Trainer(callbacks=[early_stop_callback, checkpoint_callback], logger=logger,
                         max_epochs=5)
if True:
    cpt = !ls -td nn_results/*
    cpt = cpt[0]
    print('Reading from:', cpt)
    model_dropout = mycnn_dropout.load_from_checkpoint(cpt).to(device)
else:
    model_dropout = mycnn_dropout().to(device)

#summary(model_dropout.to(device), input_size=(1, 28, 28))
summary(model_dropout, input_size=(3, 28, 28))

Global seed set to 42
/home/aparsons/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:478: LightningDeprecationWarning: Setting `Trainer(gpus=-1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=-1)` instead.
  rank_zero_deprecation(
/home/aparsons/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:589: LightningDeprecationWarning: The Trainer argument `auto_select_gpus` has been deprecated in v1.9.0 and will be removed in v2.0.0. Please use the function `pytorch_lightning.accelerators.find_usable_cuda_devices` instead.
  rank_zero_deprecation(
Auto select gpus: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Reading from: nn_results/datalab_nn_pytorch_dropout_2024-11-25T15:53.ckpt
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             896
       BatchNorm2d-2           [-1, 32, 26, 26]               0
              ReLU-3           [-1, 32, 26, 26]               0
         MaxPool2d-4           [-1, 32, 13, 13]               0
            Conv2d-5           [-1, 64, 11, 11]          18,496
              ReLU-6           [-1, 64, 11, 11]               0
         MaxPool2d-7             [-1, 64, 5, 5]               0
           Dropout-8             [-1, 64, 5, 5]               0
            Conv2d-9            [-1, 128, 3, 3]          73,856
             ReLU-10            [-1, 128, 3, 3]               0
           Linear-11                   [-1, 32]          36,896
           Linear-12                   [-1, 10]             330
Total params: 130,474
Trainab

In [10]:
myTrainer.fit(model_dropout)

/home/aparsons/.local/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:613: UserWarning: Checkpoint directory nn_results exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/aparsons/.local/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "

  | Name   | Type       | Params
--------------------------------------
0 | layer1 | Sequential | 896   
1 | layer2 | Sequential | 18.5 K
2 | layer3 | Sequential | 73.9 K
3 | fc1    | Linear     | 36.9 K
4 | fc2    | Linear     | 330   
5 | loss   | NLLLoss    | 0     
--------------------------------------
130 K     Trainable params
0         Non-trainable params
130 K     Total params
0.522     Total estimated model params size (MB)

Sanity Checking: 0it [00:00, ?it/s]

/home/aparsons/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/aparsons/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

/home/aparsons/.local/lib/python3.10/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


...

Metric val_accuracy improved by 0.004 >= min_delta = 0.001. New best score: 0.897
Epoch 2, global step 1407: 'val_accuracy' reached 0.89705 (best 0.89705), saving model to 'nn_results/datalab_nn_pytorch_dropout_2024-11-25T18:25.ckpt' as top 1


Validation: 0it [00:00, ?it/s]

Epoch 3, global step 1876: 'val_accuracy' was not in top 1


Validation: 0it [00:00, ?it/s]

Metric val_accuracy improved by 0.004 >= min_delta = 0.001. New best score: 0.901
Epoch 4, global step 2345: 'val_accuracy' reached 0.90067 (best 0.90067), saving model to 'nn_results/datalab_nn_pytorch_dropout_2024-11-25T18:25.ckpt' as top 1
`Trainer.fit` stopped: `max_epochs=5` reached.


In [12]:
#model_dropout.eval()  # Set the model to evaluation mode
## get a batch of data from the test set
#data = next(iter(test_loader))
#example_input = data[0].to(device)
#
## Get predictions
#with torch.no_grad():  # Disable gradient calculation
#    predictions = model_dropout(example_input)
#
## If you want the class with the highest probability
#predicted_class = torch.argmax(predictions, dim=1)
#
#print(predicted_class)

# ResNets
ResNets are a neural net architecture designed for image classification tasks, but they have a long (for machine learning) history dating back to the 1980s.
Generally, they are a solution to the problem that deep neural nets can "lose track" of image information in nets that aren't already well-trained. This makes it hard
for gradients to be propagated deep into the network to identify useful features. The solution was to introduce 
skip connections (residual connections, or identity maps) between stacks of convolutional layers to allow gradients to flow directly through the network, mitigating the vanishing gradient problem in very deep networks.
The residual blocks allow the model to learn identity mappings easily, making it more robust for deep architectures.

<img src="ResBlock.png" width="20%">

<img src="Original-ResNet-18-Architecture.png" width="50%">

And of course, there are many flavors of residual networks with varying depths and numbers of parameters:

<img src="resnet_param_counts.png" width="50%">

This general architecture, whereby earlier layers are fed forward with intermediate layers providing augmented context, is also very widely used, including
in U-Nets (used for image segmentation), Transformer networks (e.g. the "T" in GPT), and many others. But because we are interested in image classification,
we'll start with ResNets here.

Here's an example of some of the intermediate layers of a ResNet-18 that was trained on a very large and generic set of millions of labeled images, and is now being shown a hops berry (I think?).

<img src="resnet_feature_visual.png" width="50%">

What's interesting here is how generic the filter shapes are, especially in the first few layers.

In a previous lecture, we examined how important it was to have large training sets, and to augment our training sets with transformations in order to avoid overfitting.
Well, it turns out that training a ResNet on a huge number of images that have nothing to do with galaxies or fashion (or both?) can lead it to identify features
that are generally usefull for image classification, and these features can help jump-start galaxy classification.

Even better, someone else spend a huge amount of computing time getting you these weights, so you can leverage trainings that might have taken months on large GPU clusters to run.

Harnessing pre-trained networks for new tasks is called **learning transfer**, and it can be quite powerful.
The trick is knowning where to create the splice between your pre-trained network and a custom network that focuses on your particular task.

# PyTorch ResNet-18 (with Lightning)

In [13]:
# Version of ResNet in Lightning, paralleling structure above

class myresnet(pl.LightningModule):
    def __init__(self, num_classes=nb_classes):
        super(myresnet, self).__init__()
        
        self.model = models.resnet18(pretrained=True)
        # Freeze all layers initially (so their weights don't update)
        #for param in self.model.parameters():
        #    param.requires_grad = False

        # Modify the final fully connected layer to match the number of classes
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

        # Unfreeze specific layers (e.g., the final fully connected layer)
        #for param in self.model.fc.parameters():
        #    param.requires_grad = True

        #self.loss = nn.CrossEntropyLoss()
        self.loss = nn.NLLLoss()


    def forward(self, x):
        x = self.model(x)
        #x = F.log_softmax(x, dim=-1)
        return x

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters())
        
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode='min',
            factor=0.75,
            patience=2,
            min_lr=1e-6,
            verbose=True
        )
        
        return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "val_accuracy"}
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = self.loss(logits, y)
        self.log('train_loss', loss)
        return loss
    
    def _evaluate(self, batch, batch_idx, stage=None):
        x, y = batch
        logits = self.forward(x)
        loss = self.loss(logits, y)
        preds = torch.argmax(logits, dim=-1)
        acc = accuracy(preds, y, task="multiclass", num_classes=nb_classes)

        if stage:
            self.log(f'{stage}_loss', loss, prog_bar=True)
            self.log(f'{stage}_accuracy', acc, prog_bar=True)

        return loss, acc
    
    def validation_step(self, batch, batch_idx):
        return self._evaluate(batch, batch_idx, 'val')[0]
    
    def train_dataloader(self):
        return train_loader
    
    def val_dataloader(self):
        return test_loader

In [14]:
run_time_string = datetime.datetime.utcnow().isoformat(timespec='minutes')
filename = f'datalab_nn_pytorch_resnet_{run_time_string}'

early_stop_callback = EarlyStopping(
   monitor='val_accuracy',
   min_delta=0.001,
   patience=3,
   verbose=True,
   mode='max'
)

checkpoint_callback = ModelCheckpoint(
    monitor='val_accuracy',
    mode='max',
    dirpath='nn_results',
    filename=filename,
    verbose=True,
    save_top_k=1
)

logger = [CSVLogger("nn_results2", name=filename), TensorBoardLogger("nn_results", name=filename)]

pl.seed_everything(42)

if device == "gpu":
    myTrainer=pl.Trainer(callbacks=[early_stop_callback, checkpoint_callback], logger=logger,
                     gpus=-1, accelerator='dp', auto_select_gpus=True, max_epochs=5)
else:
    myTrainer=pl.Trainer(callbacks=[early_stop_callback, checkpoint_callback], logger=logger,
                         max_epochs=5)
    
model_resnet = myresnet().to(device)
summary(model_resnet, input_size=(3, 28, 28))

Global seed set to 42
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/aparsons/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:176: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/home/aparsons/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/aparsons/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` 

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 14, 14]           9,408
       BatchNorm2d-2           [-1, 64, 14, 14]             128
              ReLU-3           [-1, 64, 14, 14]               0
         MaxPool2d-4             [-1, 64, 7, 7]               0
            Conv2d-5             [-1, 64, 7, 7]          36,864
       BatchNorm2d-6             [-1, 64, 7, 7]             128
              ReLU-7             [-1, 64, 7, 7]               0
            Conv2d-8             [-1, 64, 7, 7]          36,864
       BatchNorm2d-9             [-1, 64, 7, 7]             128
             ReLU-10             [-1, 64, 7, 7]               0
       BasicBlock-11             [-1, 64, 7, 7]               0
           Conv2d-12             [-1, 64, 7, 7]          36,864
      BatchNorm2d-13             [-1, 64, 7, 7]             128
             ReLU-14             [-1, 6

/home/aparsons/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


In [15]:
# Slow for reason I haven't tracked down yet.
#myTrainer.fit(model_resnet)

# PyTorch ResNet-18 (without Lightning)

See https://colab.research.google.com/github/palver7/deeplearning/blob/master/FashionMNIST_with_ResNet18_no_resize.ipynb

In [16]:
# We'll could re-use the code above from previous lectures, but this is a stand-alone non-lighting version
# of a data-loader
#dataloaders = {'train': train_loader, 'val': test_loader}
#dataset_sizes = {'train': len(train_loader.dataset.images), 'val': len(test_loader.dataset.images)}

# Data augmentation and normalization for training
# Just normalization for validation
samples = 64 #num of sample per batch
data_transforms = {
    'train': transforms.Compose([
        #transforms.Resize(28),
        transforms.RandomHorizontalFlip(),
        transforms.Grayscale(3), 
        transforms.ToTensor(), 
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
    'test': transforms.Compose([
        #transforms.Resize(28),
        transforms.Grayscale(3),
        transforms.ToTensor(), 
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])
}

trainset = FashionMNIST(root='./data', train=True, 
                        download=True, transform=data_transforms['train'])

trainset, valset = random_split(trainset, (50000,10000))

trainloader = DataLoader(trainset, batch_size=samples, shuffle=True, 
                                          num_workers=4)

testset = FashionMNIST(root='./data', train=False, 
                       download=True,transform=data_transforms['test'])

testloader = DataLoader(testset, batch_size=samples, shuffle=False, 
                                         num_workers=4)
samples
valloader = DataLoader(valset, batch_size=samples, shuffle=False, 
                                         num_workers=4)

classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal',
           'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')


image_datasets = {'train': trainset, 'val': valset, 'test': testset}
dataloaders = {'train': trainloader, 'val': valloader, 'test': testloader}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [17]:
def train_model(model, criterion, optimizer, scheduler, val_acc, 
                val_loss, train_acc, train_loss,epoch, 
                num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    list = {'train': {'acc': train_acc, 'loss': train_loss}, 
        'val':{'acc': val_acc, 'loss': val_loss}}
    next = epoch
    for epoch in range(next, next+num_epochs):
        print('Epoch {}/{}'.format(epoch, next + num_epochs - 1))
        print('-' * 10)
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
        
            running_loss = 0.0
            running_corrects = 0
        
            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
        
                # zero the parameter gradients
                optimizer.zero_grad()
        
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
        
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
        
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            list[phase]['loss'].append(epoch_loss)
            list[phase]['acc'].append(epoch_acc.item())
        
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
        
        print()
        
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
        
    return model, epoch + 1

In [18]:
model = models.resnet18(pretrained=True)
#for param in model.parameters():
#    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
model.fc = nn.Linear(model.fc.in_features, nb_classes)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1) # Decay LR by a factor of 0.1 every 7 epochs

# load in previous save state, if available
if False:
    checkpoint = torch.load('./FMNIST_ResNet18_noresize.tar')
    model.load_state_dict(checkpoint['model_state_dict'])  
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    train_loss = checkpoint['train_loss']
    train_acc = checkpoint['train_acc']
    val_loss = checkpoint['val_loss']
    val_acc = checkpoint['val_acc']
    epoch = checkpoint['epoch']

In [19]:
epoch = 0
val_acc = []
val_loss = []
train_acc = []
train_loss = []

In [20]:
model, epoch = train_model(model, criterion, optimizer, scheduler, val_acc, val_loss,  train_acc, train_loss, epoch, num_epochs=5)

Epoch 0/4
----------


/home/aparsons/.local/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


train Loss: 0.4829 Acc: 0.8280
val Loss: 0.3156 Acc: 0.8870

Epoch 1/4
----------
train Loss: 0.2950 Acc: 0.8929
val Loss: 0.2787 Acc: 0.9001

Epoch 2/4
----------
train Loss: 0.2525 Acc: 0.9085
val Loss: 0.2590 Acc: 0.9057

Epoch 3/4
----------
train Loss: 0.2247 Acc: 0.9178
val Loss: 0.2358 Acc: 0.9133

Epoch 4/4
----------
train Loss: 0.2024 Acc: 0.9252
val Loss: 0.2299 Acc: 0.9159

Training complete in 2m 5s
Best val Acc: 0.915900


In [21]:
torch.save({'epoch' : epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss, 'val_loss': val_loss, 
            'train_acc': train_acc, 'val_acc': val_acc}, 
           './FMNIST_ResNet18_noresize.tar')